In [14]:
import time
import yaml
import os
import torch
from utils.IO_func import read_file_list, load_binary_file, array_to_binary_file, load_Haskins_SSR_data
from shutil import copyfile
from utils.transforms import Transform_Compose
from utils.transforms import FixMissingValues
import IPython
import matplotlib.pyplot as plt

import argparse
import pickle
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable

In [3]:
import os
from comet_ml import Experiment
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np

In [4]:
class MyLSTM(nn.Module):
    def __init__(self, D_in = 18, H = 256, D_out = 41, num_layers= 3, bidirectional=False):
        super(MyLSTM, self).__init__()
        self.hidden_dim = H
        self.num_layers = num_layers
        self.num_direction =  2 if bidirectional else 1
        self.lstm = nn.LSTM(D_in, H, num_layers, bidirectional=bidirectional)
        self.hidden2out = nn.Linear(self.num_direction*self.hidden_dim, D_out)

    def init_hidden(self, x):
        h, c = (Variable(torch.zeros(self.num_layers * self.num_direction, x.shape[1], self.hidden_dim)),
                Variable(torch.zeros(self.num_layers * self.num_direction, x.shape[1], self.hidden_dim)))
        return h, c

    def forward(self, sequence, h, c):
        output, (h, c) = self.lstm(sequence, (h, c))
        output = self.hidden2out(output)
        return output

In [5]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val

In [36]:
import argparse
import pickle
from torch.utils.data import Dataset, DataLoader

conf_dir = 'conf/SSR_conf.yaml'
buff_dir = 'current_exp'

config = yaml.load(open(conf_dir, 'r'), Loader=yaml.FullLoader)

data_path = os.path.join(buff_dir, 'data_CV')
SPK_list = ['F01']

for test_SPK in SPK_list:
    data_path_SPK = os.path.join(data_path, test_SPK)

    tr = open(os.path.join(data_path_SPK, 'train_data.pkl'), 'rb') 
    va = open(os.path.join(data_path_SPK, 'valid_data.pkl'), 'rb')        
    train_dataset, valid_dataset = pickle.load(tr), pickle.load(va)

In [37]:
def GreedyDecoder(output, labels, label_lengths, blank_label=40, collapse_repeated=True):
    
    from utils.database import PhoneTransform

    text_transform = PhoneTransform()

    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):

        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [38]:
def data_processing(data, transforms = None):
    ema = []
    labels = []
    input_lengths = []
    label_lengths = []
    
    for file_id, x, y in data:
        if transforms is not None:
            x = transforms(x)

        ema.append(torch.FloatTensor(x))
        labels.append(y)
        input_lengths.append(x.shape[0])
        label_lengths.append(len(y))
        
    ema = torch.nn.utils.rnn.pad_sequence(ema, batch_first=True)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True)        
    
    return file_id, ema, labels, input_lengths, label_lengths

In [45]:
learning_rate=2e-4
batch_size=20
epochs=80

In [46]:
hparams = {
    "n_cnn_layers": 3,
    "n_rnn_layers": 3,
    "rnn_dim": 256,
    "n_class": 41,
    "n_feats": 18,
    "stride": 2,
    "dropout": 0.1,
    "learning_rate": learning_rate,
    "batch_size": batch_size,
    "epochs": epochs
}
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                            batch_size=1,
                            shuffle=True,
                            collate_fn=lambda x: data_processing(x, None))

test_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                            batch_size=1,
                            shuffle=False,
                            collate_fn=lambda x: data_processing(x, None))

model = MyLSTM().to(device)

#print(model)
#print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
criterion = nn.CTCLoss(blank=40).to(device)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                        steps_per_epoch=int(len(train_loader)),
                                        epochs=hparams['epochs'],
                                        anneal_strategy='linear')

iter_meter = IterMeter()



In [47]:
model.train()
data_len = len(train_loader.dataset)
for epoch in range(epochs):
    for batch_idx, _data in enumerate(train_loader):
        file_id, ema, labels, input_lengths, label_lengths = _data 
        ema, labels = ema.to(device), labels.to(device)
        optimizer.zero_grad()

        
        h, c = model.init_hidden(ema)
        h, c = h.to(device), c.to(device)

        output = model(ema, h, c)  # (batch, time, n_class)
        
        output = F.log_softmax(output, dim=2)
        output = output.transpose(0, 1) # (time, batch, n_class)

        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()

        optimizer.step()
        scheduler.step()
        iter_meter.step()
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(ema), data_len,
                100. * batch_idx / len(train_loader), loss.item()))
        


Train Epoch: 0 [0/1593 (0%)]	Loss: 21.310196
Train Epoch: 0 [100/1593 (6%)]	Loss: 26.385847
Train Epoch: 0 [200/1593 (13%)]	Loss: 29.046389
Train Epoch: 0 [300/1593 (19%)]	Loss: 21.444120
Train Epoch: 0 [400/1593 (25%)]	Loss: 23.504028
Train Epoch: 0 [500/1593 (31%)]	Loss: 32.984554
Train Epoch: 0 [600/1593 (38%)]	Loss: 24.853695
Train Epoch: 0 [700/1593 (44%)]	Loss: 27.517313
Train Epoch: 0 [800/1593 (50%)]	Loss: 24.818190
Train Epoch: 0 [900/1593 (56%)]	Loss: 24.264030
Train Epoch: 0 [1000/1593 (63%)]	Loss: 14.999396
Train Epoch: 0 [1100/1593 (69%)]	Loss: 19.646482
Train Epoch: 0 [1200/1593 (75%)]	Loss: 11.727869
Train Epoch: 0 [1300/1593 (82%)]	Loss: 9.989902
Train Epoch: 0 [1400/1593 (88%)]	Loss: 6.159327
Train Epoch: 0 [1500/1593 (94%)]	Loss: 4.276491
Train Epoch: 1 [0/1593 (0%)]	Loss: 3.802938
Train Epoch: 1 [100/1593 (6%)]	Loss: 3.601332
Train Epoch: 1 [200/1593 (13%)]	Loss: 3.447984
Train Epoch: 1 [300/1593 (19%)]	Loss: 3.350161
Train Epoch: 1 [400/1593 (25%)]	Loss: 3.540484
Tr

Train Epoch: 10 [1400/1593 (88%)]	Loss: 3.052051
Train Epoch: 10 [1500/1593 (94%)]	Loss: 3.280005
Train Epoch: 11 [0/1593 (0%)]	Loss: 3.361163
Train Epoch: 11 [100/1593 (6%)]	Loss: 3.287887
Train Epoch: 11 [200/1593 (13%)]	Loss: 3.021090
Train Epoch: 11 [300/1593 (19%)]	Loss: 3.280925
Train Epoch: 11 [400/1593 (25%)]	Loss: 3.056537
Train Epoch: 11 [500/1593 (31%)]	Loss: 3.116244
Train Epoch: 11 [600/1593 (38%)]	Loss: 3.293427
Train Epoch: 11 [700/1593 (44%)]	Loss: 3.248296
Train Epoch: 11 [800/1593 (50%)]	Loss: 3.076493
Train Epoch: 11 [900/1593 (56%)]	Loss: 3.450681
Train Epoch: 11 [1000/1593 (63%)]	Loss: 3.423156
Train Epoch: 11 [1100/1593 (69%)]	Loss: 3.207210
Train Epoch: 11 [1200/1593 (75%)]	Loss: 2.896209
Train Epoch: 11 [1300/1593 (82%)]	Loss: 3.479307
Train Epoch: 11 [1400/1593 (88%)]	Loss: 2.818757
Train Epoch: 11 [1500/1593 (94%)]	Loss: 2.768070
Train Epoch: 12 [0/1593 (0%)]	Loss: 2.840521
Train Epoch: 12 [100/1593 (6%)]	Loss: 2.897292
Train Epoch: 12 [200/1593 (13%)]	Loss: 2

Train Epoch: 21 [900/1593 (56%)]	Loss: 2.143648
Train Epoch: 21 [1000/1593 (63%)]	Loss: 2.110667
Train Epoch: 21 [1100/1593 (69%)]	Loss: 1.952557
Train Epoch: 21 [1200/1593 (75%)]	Loss: 2.087937
Train Epoch: 21 [1300/1593 (82%)]	Loss: 2.157031
Train Epoch: 21 [1400/1593 (88%)]	Loss: 2.077457
Train Epoch: 21 [1500/1593 (94%)]	Loss: 1.945209
Train Epoch: 22 [0/1593 (0%)]	Loss: 2.059566
Train Epoch: 22 [100/1593 (6%)]	Loss: 2.104927
Train Epoch: 22 [200/1593 (13%)]	Loss: 1.695912
Train Epoch: 22 [300/1593 (19%)]	Loss: 1.869347
Train Epoch: 22 [400/1593 (25%)]	Loss: 1.968154
Train Epoch: 22 [500/1593 (31%)]	Loss: 1.969347
Train Epoch: 22 [600/1593 (38%)]	Loss: 1.904996
Train Epoch: 22 [700/1593 (44%)]	Loss: 2.018320
Train Epoch: 22 [800/1593 (50%)]	Loss: 2.541573
Train Epoch: 22 [900/1593 (56%)]	Loss: 2.289386
Train Epoch: 22 [1000/1593 (63%)]	Loss: 2.047191
Train Epoch: 22 [1100/1593 (69%)]	Loss: 1.938551
Train Epoch: 22 [1200/1593 (75%)]	Loss: 1.867007
Train Epoch: 22 [1300/1593 (82%)]	L

Train Epoch: 32 [400/1593 (25%)]	Loss: 1.512898
Train Epoch: 32 [500/1593 (31%)]	Loss: 1.764148
Train Epoch: 32 [600/1593 (38%)]	Loss: 1.556425
Train Epoch: 32 [700/1593 (44%)]	Loss: 1.368546
Train Epoch: 32 [800/1593 (50%)]	Loss: 1.643224
Train Epoch: 32 [900/1593 (56%)]	Loss: 1.965700
Train Epoch: 32 [1000/1593 (63%)]	Loss: 1.945158
Train Epoch: 32 [1100/1593 (69%)]	Loss: 2.030196
Train Epoch: 32 [1200/1593 (75%)]	Loss: 1.666659
Train Epoch: 32 [1300/1593 (82%)]	Loss: 1.812455
Train Epoch: 32 [1400/1593 (88%)]	Loss: 1.498348
Train Epoch: 32 [1500/1593 (94%)]	Loss: 1.702220
Train Epoch: 33 [0/1593 (0%)]	Loss: 1.437135
Train Epoch: 33 [100/1593 (6%)]	Loss: 1.746848
Train Epoch: 33 [200/1593 (13%)]	Loss: 2.206249
Train Epoch: 33 [300/1593 (19%)]	Loss: 1.431959
Train Epoch: 33 [400/1593 (25%)]	Loss: 1.935187
Train Epoch: 33 [500/1593 (31%)]	Loss: 1.639541
Train Epoch: 33 [600/1593 (38%)]	Loss: 2.221561
Train Epoch: 33 [700/1593 (44%)]	Loss: 1.604845
Train Epoch: 33 [800/1593 (50%)]	Loss:

Train Epoch: 42 [1500/1593 (94%)]	Loss: 1.529582
Train Epoch: 43 [0/1593 (0%)]	Loss: 2.198587
Train Epoch: 43 [100/1593 (6%)]	Loss: 1.876735
Train Epoch: 43 [200/1593 (13%)]	Loss: 1.201084
Train Epoch: 43 [300/1593 (19%)]	Loss: 1.478490
Train Epoch: 43 [400/1593 (25%)]	Loss: 1.873940
Train Epoch: 43 [500/1593 (31%)]	Loss: 1.535146
Train Epoch: 43 [600/1593 (38%)]	Loss: 1.885857
Train Epoch: 43 [700/1593 (44%)]	Loss: 1.530355
Train Epoch: 43 [800/1593 (50%)]	Loss: 1.761753
Train Epoch: 43 [900/1593 (56%)]	Loss: 1.641060
Train Epoch: 43 [1000/1593 (63%)]	Loss: 1.807918
Train Epoch: 43 [1100/1593 (69%)]	Loss: 1.932219
Train Epoch: 43 [1200/1593 (75%)]	Loss: 1.756053
Train Epoch: 43 [1300/1593 (82%)]	Loss: 1.535158
Train Epoch: 43 [1400/1593 (88%)]	Loss: 1.635136
Train Epoch: 43 [1500/1593 (94%)]	Loss: 1.652480
Train Epoch: 44 [0/1593 (0%)]	Loss: 1.625776
Train Epoch: 44 [100/1593 (6%)]	Loss: 1.355201
Train Epoch: 44 [200/1593 (13%)]	Loss: 1.650342
Train Epoch: 44 [300/1593 (19%)]	Loss: 1.

Train Epoch: 53 [1000/1593 (63%)]	Loss: 1.526792
Train Epoch: 53 [1100/1593 (69%)]	Loss: 1.624243
Train Epoch: 53 [1200/1593 (75%)]	Loss: 1.445471
Train Epoch: 53 [1300/1593 (82%)]	Loss: 1.738680
Train Epoch: 53 [1400/1593 (88%)]	Loss: 1.484964
Train Epoch: 53 [1500/1593 (94%)]	Loss: 1.893504
Train Epoch: 54 [0/1593 (0%)]	Loss: 1.215502
Train Epoch: 54 [100/1593 (6%)]	Loss: 1.522900
Train Epoch: 54 [200/1593 (13%)]	Loss: 1.484900
Train Epoch: 54 [300/1593 (19%)]	Loss: 1.474905
Train Epoch: 54 [400/1593 (25%)]	Loss: 1.338569
Train Epoch: 54 [500/1593 (31%)]	Loss: 1.353220
Train Epoch: 54 [600/1593 (38%)]	Loss: 1.610574
Train Epoch: 54 [700/1593 (44%)]	Loss: 2.036650
Train Epoch: 54 [800/1593 (50%)]	Loss: 1.478721
Train Epoch: 54 [900/1593 (56%)]	Loss: 1.547753
Train Epoch: 54 [1000/1593 (63%)]	Loss: 1.625532
Train Epoch: 54 [1100/1593 (69%)]	Loss: 1.376763
Train Epoch: 54 [1200/1593 (75%)]	Loss: 2.113838
Train Epoch: 54 [1300/1593 (82%)]	Loss: 1.172675
Train Epoch: 54 [1400/1593 (88%)]	

Train Epoch: 64 [500/1593 (31%)]	Loss: 1.553845
Train Epoch: 64 [600/1593 (38%)]	Loss: 1.411383
Train Epoch: 64 [700/1593 (44%)]	Loss: 1.176699
Train Epoch: 64 [800/1593 (50%)]	Loss: 1.796999
Train Epoch: 64 [900/1593 (56%)]	Loss: 1.251201
Train Epoch: 64 [1000/1593 (63%)]	Loss: 1.566220
Train Epoch: 64 [1100/1593 (69%)]	Loss: 1.581422
Train Epoch: 64 [1200/1593 (75%)]	Loss: 1.345609
Train Epoch: 64 [1300/1593 (82%)]	Loss: 1.280065
Train Epoch: 64 [1400/1593 (88%)]	Loss: 1.563831
Train Epoch: 64 [1500/1593 (94%)]	Loss: 1.612743
Train Epoch: 65 [0/1593 (0%)]	Loss: 1.871540
Train Epoch: 65 [100/1593 (6%)]	Loss: 1.659234
Train Epoch: 65 [200/1593 (13%)]	Loss: 1.647990
Train Epoch: 65 [300/1593 (19%)]	Loss: 1.278894
Train Epoch: 65 [400/1593 (25%)]	Loss: 1.485899
Train Epoch: 65 [500/1593 (31%)]	Loss: 1.639910
Train Epoch: 65 [600/1593 (38%)]	Loss: 1.429198
Train Epoch: 65 [700/1593 (44%)]	Loss: 1.699653
Train Epoch: 65 [800/1593 (50%)]	Loss: 1.605219
Train Epoch: 65 [900/1593 (56%)]	Loss:

Train Epoch: 75 [0/1593 (0%)]	Loss: 1.420161
Train Epoch: 75 [100/1593 (6%)]	Loss: 1.393553
Train Epoch: 75 [200/1593 (13%)]	Loss: 1.589228
Train Epoch: 75 [300/1593 (19%)]	Loss: 1.306402
Train Epoch: 75 [400/1593 (25%)]	Loss: 1.554134
Train Epoch: 75 [500/1593 (31%)]	Loss: 1.509404
Train Epoch: 75 [600/1593 (38%)]	Loss: 1.630840
Train Epoch: 75 [700/1593 (44%)]	Loss: 1.359795
Train Epoch: 75 [800/1593 (50%)]	Loss: 1.226352
Train Epoch: 75 [900/1593 (56%)]	Loss: 1.306952
Train Epoch: 75 [1000/1593 (63%)]	Loss: 1.457082
Train Epoch: 75 [1100/1593 (69%)]	Loss: 1.149705
Train Epoch: 75 [1200/1593 (75%)]	Loss: 1.293676
Train Epoch: 75 [1300/1593 (82%)]	Loss: 1.399068
Train Epoch: 75 [1400/1593 (88%)]	Loss: 1.553788
Train Epoch: 75 [1500/1593 (94%)]	Loss: 1.507393
Train Epoch: 76 [0/1593 (0%)]	Loss: 0.930600
Train Epoch: 76 [100/1593 (6%)]	Loss: 1.552244
Train Epoch: 76 [200/1593 (13%)]	Loss: 1.457642
Train Epoch: 76 [300/1593 (19%)]	Loss: 1.244537
Train Epoch: 76 [400/1593 (25%)]	Loss: 1.3

In [48]:
model.eval()
pred = []
label = []

from jiwer import wer

for batch_idx, _data in enumerate(test_loader):
    fid, ema, labels, input_lengths, label_lengths = _data 
    ema, labels = ema.to(device), labels.to(device)

    optimizer.zero_grad()

    h, c = model.init_hidden(ema)
    h, c = h.to(device), c.to(device)

    output = model(ema, h, c)  # (batch, time, n_class)

    output = F.log_softmax(output, dim=2)
    output = output.transpose(0, 1) # (time, batch, n_class)
    
    loss = criterion(output, labels, input_lengths, label_lengths)
  #  test_loss += loss.item() / len(test_loader)

    decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)

    
    pred.append(' '.join(decoded_preds[0]))
    label.append(' '.join(decoded_targets[0]))
    
print(len(pred))
print(len(label))

error = wer(pred, label)
print(error)

50
50
3.149390243902439


In [49]:
for i in range(len(pred)):
    print(pred[i])
    print(label[i])
    print('#####################')

AY L L EY
SP S M OW K IY F AY ER Z L AE K F L EY M AE N D SP SHH IY T SP
#####################
AH K EY R M DH
SP DH AH S AO L T B R IY Z K EY M AH K R AO S F ER M DH AH S IY SP
#####################
AH L D R K R S ER M DH DH
SP DH AH S AO L T B R IY Z K EY M SP AH K R AO S F ER M DH AH S IY SP
#####################
DH AH R K R AO S ER AH M DH DH
SP DH AH S AO L T B R IY Z K EY M AH K R AO S F ER M DH AH S IY SP
#####################
AH OW DH AH AH L AH AH P N
SP DH AH G ER L AE T DH AH B UW TH S OW L D F IH F T IY B   N D Z SP
#####################
AH ER AH AH DH L V
SP DH AH G ER L AE T DH AH B UW TH S OW L D F IH F T IY B   N D Z SP
#####################
AH ER AH DH L AH
SP DH AH G ER L AE T DH AH B UW TH S OW L D F IH F T IY B   N D Z SP
#####################
DH AH M N L S S
SP DH AH S M AO L P AH P N AO D AH SHH OW L IH N DH AH S   K SP
#####################
AH N DH AH S
SP DH AH S M AO L P AH P SP N AO D AH SHH OW L IH N DH AH S   K SP
#####################
DH AH N L DH AH S S
SP 